In [1]:
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path

dataset_path = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\parquet_data_2")
out_csv = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_28_02.csv")

# Force hive partition discovery so MMSI/Segment come in as columns
dataset = ds.dataset(dataset_path, format="parquet", partitioning="hive")
table = dataset.to_table()  # you can also add filter=... here
df = table.to_pandas()

# (Optional) ensure MMSI is string
df["MMSI"] = df["MMSI"].astype(str)

df.to_csv(out_csv, index=False)
print(df.columns.tolist(), len(df))


['Timestamp', 'Latitude', 'Longitude', 'SOG', 'COG', 'MMSI', 'Segment'] 6614370


keep vessels from 00:00:00 to 12:00:00

In [3]:
import pandas as pd
from pathlib import Path

# --- CONFIG ---
csv_path = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_28_02.csv")
output_path = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_until12.csv")


# --- READ ---
df = pd.read_csv(csv_path)

# Convert 'Timestamp' column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Keep only rows where the time is between 00:00:00 and 12:00:00 (inclusive)
mask = (df['Timestamp'].dt.hour < 12) | (
    (df['Timestamp'].dt.hour == 12) & 
    (df['Timestamp'].dt.minute.eq(0)) & 
    (df['Timestamp'].dt.second.eq(0))
)
df_filtered = df[mask]

# --- SAVE ---
df_filtered.to_csv(output_path, index=False)

print(f"✅ Kept {len(df_filtered)} rows (00:00:00 → 12:00:00).")
print(f"💾 Saved to: {output_path}")


✅ Kept 3177925 rows (00:00:00 → 12:00:00).
💾 Saved to: C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_until12.csv


keep only the vessels with id's in 27-02

In [1]:
import pandas as pd
from pathlib import Path

# --- CONFIG ---
file_1 = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_until12.csv")  # file to filter
file_2 = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\data\ais_combined.csv")  # file with reference MMSIs
output = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_until12_filtered.csv")

# --- READ FILES ---
df1 = pd.read_csv(file_1)   # the one you want to filter
df2 = pd.read_csv(file_2)   # the reference file

# --- FILTER: keep only rows where MMSI exists in both ---
df_filtered = df1[df1["MMSI"].isin(df2["MMSI"])]

# --- SAVE ---
df_filtered.to_csv(output, index=False)

print(f"✅ Filtered {len(df1)} → {len(df_filtered)} rows.")
print(f"💾 Saved to: {output}")


✅ Filtered 3177925 → 2119326 rows.
💾 Saved to: C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_until12_filtered.csv


In [4]:
import pandas as pd
from pathlib import Path

# --- CONFIG ---
file1 = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\data\ais_combined.csv")
file2 = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\ais_combined_until12_filtered.csv")
output = Path(r"C:\Users\Nikos\Desktop\DTU\Deep Learning\project\data\ais_combined_merged.csv")

# --- READ ---
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# --- MERGE (append vertically) ---
df_merged = pd.concat([df1, df2], ignore_index=True)

# --- SAVE ---
df_merged.to_csv(output, index=False)
print(f"✅ Merged {len(df1)} + {len(df2)} = {len(df_merged)} rows")
print(f"💾 Saved to: {output}")


✅ Merged 5242595 + 2119326 = 7361921 rows
💾 Saved to: C:\Users\Nikos\Desktop\DTU\Deep Learning\project\data\ais_combined_merged.csv
